In [1]:
import numpy as np
import pandas as pd

In [2]:
rating_file ='../data/ml-20m/'+'ratings.csv'

In [3]:
rating_pd = pd.read_csv(rating_file)
rating_pd.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
rating_pd .to_csv('../data/ratings.txt', sep='\t', index=False, header=False)

In [5]:
rating_np = np.loadtxt('../data/ratings.txt', delimiter='\t',dtype=np.int64)

C:\Users\24717\AppData\Local\Temp\ipykernel_16252\2709960758.py:1: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  rating_np = np.loadtxt('../data/ratings.txt', delimiter='\t',dtype=np.int64)


In [6]:
print(rating_np[:10])

[[         1          2          3 1112486027]
 [         1         29          3 1112484676]
 [         1         32          3 1112484819]
 [         1         47          3 1112484727]
 [         1         50          3 1112484580]
 [         1        112          3 1094785740]
 [         1        151          4 1094785734]
 [         1        223          4 1112485573]
 [         1        253          4 1112484940]
 [         1        260          4 1112484826]]


In [7]:
###定义一个分离正负样本的函数，平均评分为分界线
def classify_ratings(data):
    # 将数据转换为DataFrame以便于处理
    df = pd.DataFrame(data, columns=['UserID', 'MovieID', 'Rating', 'Timestamp'])

    # 计算每个用户的平均评分
    user_avg_ratings = df.groupby('UserID')['Rating'].mean().to_dict()

    # 初始化正样本和负样本列表
    positive_samples = []
    negative_samples = []

    # 遍历数据，分类正负样本
    for index, row in df.iterrows():
        user_id = row['UserID']
        rating = row['Rating']
        avg_rating = user_avg_ratings[user_id]

        # 判断是正样本还是负样本
        if rating > avg_rating:
            positive_samples.append([user_id, row['MovieID'], rating, row['Timestamp']])
        else:
            negative_samples.append([user_id, row['MovieID'], rating, row['Timestamp']])

    return positive_samples, negative_samples

In [8]:
#positive_samples, negative_samples = classify_ratings(rating_np)
#np.save('../data/data_mid/positive_samples.npy', positive_samples)
#np.save('../data/data_mid/negative_samples.npy', negative_samples)
###保存正负数据

In [9]:
########################################划分数据集#####################################
def dataset_split(rating_np, ratio):
    print('splitting dataset ...')

    # train:eval:test = 6:2:2
    eval_ratio = 0.2
    test_ratio = 0.2
    n_ratings = rating_np.shape[0]

    eval_indices = np.random.choice(list(range(n_ratings)), size=int(n_ratings * eval_ratio), replace=False)
    left = set(range(n_ratings)) - set(eval_indices)
    test_indices = np.random.choice(list(left), size=int(n_ratings * test_ratio), replace=False)
    train_indices = list(left - set(test_indices))
    if ratio < 1:
        train_indices = np.random.choice(list(train_indices), size=int(len(train_indices) * ratio), replace=False)

    train_data = rating_np[train_indices]
    eval_data = rating_np[eval_indices]
    test_data = rating_np[test_indices]

    return train_data, eval_data, test_data

In [10]:
ratio = 0.8
train_data, eval_data, test_data = dataset_split(rating_np, ratio)

splitting dataset ...


In [11]:
np.save('../data/data_mid/train_data.npy', train_data)
np.save('../data/data_mid/eval_data.npy', eval_data)
np.save('../data/data_mid/test_data.npy', test_data)

In [12]:
n_user = max(rating[0] for rating in train_data)
n_movie = max(rating[1] for rating in train_data)
print(train_data.shape)
print(n_user)       ##138493
print(n_movie)      ##131262

(9600127, 4)
138493
131262


In [13]:
###随机初始化向量函数
def initialize_embeddings(num_users, num_movies, num_adj , embedding_dim, seed=42):
    """
    初始化用户和电影的嵌入向量。
    
    参数:
    num_users (int): 用户数量。
    num_movies (int): 电影数量。
    embedding_dim (int): 嵌入向量的维度。
    seed (int): 随机种子，用于复现结果。
    
    返回:
    user_embeddings (numpy.ndarray): 用户嵌入向量。
    movie_embeddings (numpy.ndarray): 电影嵌入向量。
    """
    np.random.seed(seed)  # 设置随机种子
    # 使用正态分布初始化用户嵌入向量
    user_embeddings = np.random.normal(scale=0.1, size=(num_users, embedding_dim))
    adj_embeddings = np.random.normal(scale=0.1, size=(num_adj, embedding_dim))
    # 使用正态分布初始化电影嵌入向量
    movie_embeddings = np.random.normal(scale=0.1, size=(num_movies, embedding_dim))
    
    return user_embeddings, movie_embeddings , adj_embeddings

In [14]:
genres_file ='../data/ml-20m/'+'movies.csv'

In [15]:
genres = pd.read_csv(genres_file)
genres.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
genres = genres.drop('title', axis=1)
genres.head()

,movieId,genres
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


In [17]:
def create_movie_category_dict(df):
    """
    从DataFrame中创建一个电影ID到类别集合的字典。
    参数:
    df (pd.DataFrame): 包含电影ID和类别的数据帧。
    返回:
    dict: 电影ID映射到类别集合的字典。
    """
    kg = {}
    for index, row in df.iterrows():
        movie_id = row['movieId']  # 假设列名是'movieid'
        categories = row['genres']  # 假设列名是'categories'
        # 使用set来转换类别字符串为集合，并去除空格
        if categories == "(no genres listed)":
            continue  # 如果没有类别，跳过这一行
        kg[movie_id] = {category.strip().lower() for category in categories.split('|')}
    return kg

In [18]:
kg = create_movie_category_dict(genres)

In [19]:
print(kg[1])

{'animation', 'adventure', 'children', 'fantasy', 'comedy'}


In [20]:
n_adj=0
for movie in kg:
    n_adj = n_adj + len(kg[movie])
print(n_adj)

54160


In [21]:
embedding_dim = 25
user_embeddings, movie_embeddings , adj_embeddings = initialize_embeddings(n_user, n_movie+2, n_adj,embedding_dim, seed=42) 

In [22]:
print(adj_embeddings.shape)

(54160, 25)


In [23]:
from torch import nn
import torch
device = torch.device('cuda:0')

In [24]:
tensor_data_from_train_data = torch.as_tensor(train_data)
tensor_data_from_eval_data = torch.as_tensor(eval_data)
tensor_data_from_test_data = torch.as_tensor(test_data)
tensor_data_from_train_data = tensor_data_from_train_data.to(device)
tensor_data_from_eval_data = tensor_data_from_eval_data.to(device)
tensor_data_from_test_data = tensor_data_from_test_data.to(device)
print(tensor_data_from_train_data[1])

tensor([     42583,       6657,          5, 1071604865], device='cuda:0')


In [25]:
class Net(nn.Module):
    def __init__(self,n_users,n_movies,n_adj,embedding_dim):
        super(Net,self).__init__()
        self.num_users  = n_users
        self.num_movies = n_movies
        self.num_adj = n_adj
        self.embedding_dim = embedding_dim
        
        self.__init_weight()
        
        self.model = nn.Sequential(
            
        )
    def __init_weight(self):
        self.embedding_user = torch.nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.embedding_dim)
        self.embedding_movie = torch.nn.Embedding(
            num_embeddings=self.num_movies, embedding_dim=self.embedding_dim)
        print("using Normal distribution N(0,1) initialization for embedding")
        
    def forward(self):
        
        return self.embedding_user,self.embedding_movie

In [26]:
net = Net(n_user, n_movie, n_adj, embedding_dim)
net.to(device)
embedding_user,embedding_movie = net()
print(embedding_user)


using Normal distribution N(0,1) initialization for embedding
Embedding(138493, 25)
